In [2]:
import pandas as pd
import numpy as np
import math
data=pd.read_csv('SPECT.csv')
data=np.array(data)
np.random.shuffle(data)

In [3]:
n_features=len(data[0])-1
n_data=len(data)

In [4]:
#Iris-setosa
for i in range(n_data):
    if(data[i][n_features]=="Yes"):
        data[i][n_features]=1
    else:
        data[i][n_features]=0

In [ ]:
thresold=0.5
n_epochs=1000
learning_rate=5
k_fold=10
layer=[n_features,5,1]
eps=0.001
weights=[[[1/(layer[i]+layer[i+1])]*layer[i] for j in range(layer[i+1])] for i in range(len(layer)-1)]
bias=[[1]*layer[i+1] for i in range(len(layer)-1)]
nodes=[[0]*layer[i] for i in range(len(layer))]
error=[[0]*layer[i] for i in range(len(layer))]

def function(row,nodes,weights,bias,thresold,layer):
    for i in range(len(row)):
        nodes[0][i]=row[i]
    for lay in range(1,len(layer)):
        for j in range(layer[lay]):
            y=bias[lay-1][j]
            for k in weights[lay-1][j]:
                y=y+nodes[lay-1][j]*k
            nodes[lay][j]=1/(1+math.exp(-y))
#     print(nodes[len(layer)-1])
    if(nodes[len(layer)-1][0]>=thresold):
        y=1
    else:
        y=0
    return nodes,y

def valid(z,cost,a,b,c,d):
    if cost==0:
        if z==1:
            return a+1,b,c,d
        else:
            return a,b,c+1,d
    else:
        if z==0:
            return a,b+1,c,d
        else:
            return a,b,c,d+1

def kfolddata(j,data):
    x_test=data[j*k_fold:(j+1)*k_fold,0:n_features]
    y_test=data[j*k_fold:(j+1)*k_fold,n_features]
    if(j+2<=k_fold):
        x_train=data[(j+1)*k_fold:,0:n_features]
        y_train=data[(j+1)*k_fold:,n_features]
        if(j-1>=0):
            x_train=np.append(x_train,data[0:j*k_fold,0:n_features],axis=0)
            y_train=np.append(y_train,data[0:j*k_fold,n_features],axis=0)
    else:
        x_train=data[0:j*k_fold,0:n_features]
        y_train=data[0:j*k_fold,n_features]
    return x_train,y_train,x_test,y_test
    
def back_prop(weights,nodes,row,thresold,bias,learning_rate,cost,error,z):
    for i in range(len(row)):
        nodes[0][i]=row[i]
    #output layer error
    error[len(error)-1][0]=(z-nodes[len(error)-1][0])*(1-nodes[len(error)-1][0])*nodes[len(error)-1][0]
#     print(error[len(error)-1][0])
    #hidden layers
    for lay in range(len(error)-2,0,-1):
        for nod in range(len(nodes[lay])):
            err=0
            for nxtlaynod in range(len(nodes[lay+1])):
                err+=error[lay+1][nxtlaynod]*weights[lay][nxtlaynod][nod]             
            error[lay][nod]=nodes[lay][nod]*(1-nodes[lay][nod])*err*-1
    #update weights
    for lay in range(len(weights)):
        for j in range(len(weights[lay])):
            for i in range(len(weights[lay][j])):
                weights[lay][j][i]+=learning_rate*nodes[lay][i]*error[lay+1][j]
    for lay in range(len(bias)):
        for i in range(len(bias[lay])):
            bias[lay][i]+=learning_rate*error[lay+1][i]
    return weights,bias

def printfoldresult(test_TT,test_TF,test_FF,test_FT,eps):
    print("trainTT:",train_TT," trainTF:",train_TF," trainFF:",train_FF," trainFT:",train_FT)
    print("train_accuracy",(train_TT+train_FF)/(train_TT+train_TF+train_FF+train_FT+eps))
    print("train Precision +:",(train_TT)/(train_TT+train_FT+eps))
    print("train Precision -:",(train_FF)/(train_FF+train_TF+eps))
    print("train recall +:",(train_TT)/(train_TT+train_TF+eps))
    print("train recall -:",(train_FF)/(train_FF+train_FT+eps))
    print("testTT:",test_TT," testTF:",test_TF," testFF:",test_FF," testFT:",test_FT)
    print("test_accuracy",(test_TT+test_FF)/(test_TT+test_TF+test_FF+test_FT+eps))
    print("test Precision +:",(test_TT)/(test_TT+test_FT+eps))
    print("test Precision -:",(test_FF)/(test_FF+test_TF+eps))
    print("test recall +:",(test_TT)/(test_TT+test_TF+eps))
    print("test recall -:",(test_FF)/(test_FF+test_FT+eps))

def averageresults(train_TT,train_TF,train_FF,train_FT,test_TT,test_TF,test_FF,test_FT,eps,accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg):
    accTra+=(train_TT+train_FF)/(train_TT+train_TF+train_FF+train_FT+eps)
    precTraPos+=((train_TT)/(train_TT+train_FT+eps))
    precTraNeg+=((train_FF)/(train_FF+train_TF+eps))
    recTraPos+=((train_TT)/(train_TT+train_TF+eps))
    recTraNeg+=((train_FF)/(train_FF+train_FT+eps))
    accTes+=((test_TT+test_FF)/(test_TT+test_TF+test_FF+test_FT+eps))
    precTesPos+=((test_TT)/(test_TT+test_FT+eps))
    precTesNeg+=((test_FF)/(test_FF+test_TF+eps))
    recTesPos+=((test_TT)/(test_TT+test_TF+eps))
    recTesNeg+=((test_FF)/(test_FF+test_FT+eps))
    return accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg 

def printepochresult(accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg,k_fold):
    print("average result for ",k_fold,"fold")
    print("train_accuracy:",accTra/k_fold)
    print("train Precision +:",precTraPos/k_fold)
    print("train Precision -:",precTraNeg/k_fold)
    print("train recall +:",recTraPos/k_fold)
    print("train recall -:",recTraNeg/k_fold)
    print("test_accuracy",accTes/k_fold)
    print("test Precision +:",precTesPos/k_fold)
    print("test Precision -:",precTesNeg/k_fold)
    print("test recall +:",recTesPos/k_fold)
    print("test recall -:",recTesNeg/k_fold)
    
for i in range(n_epochs):
    print("epoch no",i+1,"+"*60)
    accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg=(0 for i in range(10))
    for j in range(int(n_data/k_fold)):
#         print("fold no:",j)
        x_train,y_train,x_test,y_test=kfolddata(j,data)
        #validation variables
        train_TT,train_TF,train_FF,train_FT,test_TT,test_TF,test_FF,test_FT=(0 for i in range(8))
        for i in range(len(x_train)):
            nodes,y=function(x_train[i],nodes,weights,bias,thresold,layer)
            z=y_train[i]
            cost=z-y
            train_TT,train_TF,train_FF,train_FT=valid(y,cost,train_TT,train_TF,train_FF,train_FT)
            weights,bias=back_prop(weights,nodes,x_train[i],thresold,bias,learning_rate,cost,error,z)
#             print(weights)
        for i in range(len(x_test)):
            nodes,y=function(x_test[i],nodes,weights,bias,thresold,layer)
            z=y_test[i]
            cost=z-y
            test_TT,test_TF,test_FF,test_FT=valid(y,cost,test_TT,test_TF,test_FF,test_FT)
#         printfoldresult(test_TT,test_TF,test_FF,test_FT,eps)
        accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg=averageresults(train_TT,train_TF,train_FF,train_FT,test_TT,test_TF,test_FF,test_FT,eps,accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg)
    printepochresult(accTra,accTes,precTraPos,precTraNeg,recTraPos,recTraNeg,precTesPos,precTesNeg,recTesPos,recTesNeg,int(n_data/k_fold))


epoch no 1 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7420826601985115
train Precision +: 0.7925138667627345
train Precision -: 0.13964479101642177
train recall +: 0.9160153862966668
train recall -: 0.05512481552622943
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 2 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7428763925975171
train Precision +: 0.7929187246961796
train Precision -: 0.14295516181878598
train recall +: 0.9166921929653132
train recall -: 0.05651423356011793
test_accuracy 0.765308084576158
test Precision +: 0.7614623153069311
test Precision -: 0.003845769269226924
test recall +: 0.9614125019567914
test recall -: 0.03842311534619228
epoch no 3 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average

average result for  26 fold
train_accuracy: 0.744545371435318
train Precision +: 0.7924516435658462
train Precision -: 0.13365481759347297
train recall +: 0.9204265026799037
train recall -: 0.049957460498067444
test_accuracy 0.7037757762685272
test Precision +: 0.6922384684608462
test Precision -: 0.011537307807680771
test recall +: 0.8844979710926995
test recall -: 0.11526934603857683
epoch no 20 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7421559004623367
train Precision +: 0.7916719049928728
train Precision -: 0.1270679690624109
train recall +: 0.9177642703130549
train recall -: 0.04852946208237914
test_accuracy 0.7037757762685272
test Precision +: 0.6922384684608462
test Precision -: 0.011537307807680771
test recall +: 0.8844979710926995
test recall -: 0.11526934603857683
epoch no 21 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.743399476702398
train Precision

average result for  26 fold
train_accuracy: 0.7536266102065995
train Precision +: 0.7940609300297994
train Precision -: 0.14677754809723098
train recall +: 0.9327875831529908
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 38 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7533272997878733
train Precision +: 0.7938965651832324
train Precision -: 0.14510537999751627
train recall +: 0.9325981183796078
train recall -: 0.04572341468445346
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 39 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7534769549972364
train Precisi

average result for  26 fold
train_accuracy: 0.7536266102065995
train Precision +: 0.7940609300297994
train Precision -: 0.14677754809723098
train recall +: 0.9327875831529908
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 56 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7536266102065995
train Precision +: 0.7940609300297994
train Precision -: 0.14677754809723098
train recall +: 0.9327875831529908
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 57 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7536266102065995
train Precisi

average result for  26 fold
train_accuracy: 0.7536266102065995
train Precision +: 0.7940609300297994
train Precision -: 0.14677754809723098
train recall +: 0.9327875831529908
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 74 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7536266102065995
train Precision +: 0.7940609300297994
train Precision -: 0.14677754809723098
train recall +: 0.9327875831529908
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 75 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7536266102065995
train Precisi

...

average result for  26 fold
train_accuracy: 0.7533272997878733
train Precision +: 0.793991313269978
train Precision -: 0.14643418749115272
train recall +: 0.9324086536062246
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 560 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7533272997878733
train Precision +: 0.793991313269978
train Precision -: 0.14643418749115272
train recall +: 0.9324086536062246
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 561 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7533272997878733
train Precisi

average result for  26 fold
train_accuracy: 0.7533272997878733
train Precision +: 0.793991313269978
train Precision -: 0.14643418749115272
train recall +: 0.9324086536062246
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 578 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7533272997878733
train Precision +: 0.793991313269978
train Precision -: 0.14643418749115272
train recall +: 0.9324086536062246
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 579 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7533272997878733
train Precisi

average result for  26 fold
train_accuracy: 0.753027989369147
train Precision +: 0.7939200923812844
train Precision -: 0.1460837896265173
train recall +: 0.9320297240594587
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 596 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7531776445785101
train Precision +: 0.79368419475418
train Precision -: 0.14093696804319578
train recall +: 0.9327875831529908
train recall -: 0.04429893963911954
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 597 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7533272997878733
train Precision

average result for  26 fold
train_accuracy: 0.7531776445785101
train Precision +: 0.7939560697871314
train Precision -: 0.14629280193013566
train recall +: 0.9322191888328416
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 614 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7531776445785101
train Precision +: 0.7939560697871314
train Precision -: 0.14629280193013566
train recall +: 0.9322191888328416
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 615 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.7531776445785101
train Preci

average result for  26 fold
train_accuracy: 0.7527286789504206
train Precision +: 0.7935772865922579
train Precision -: 0.14103817223108264
train recall +: 0.9322191888328416
train recall -: 0.04429893963911954
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 632 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.753027989369147
train Precision +: 0.7939205313809761
train Precision -: 0.14616712500319637
train recall +: 0.9320297240594587
train recall -: 0.04643565220712042
test_accuracy 0.7345419304223426
test Precision +: 0.7268503918838888
test Precision -: 0.007691538538453848
test recall +: 0.9229552365247454
test recall -: 0.07684623069238455
epoch no 633 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  26 fold
train_accuracy: 0.753027989369147
train Precisi